In [1]:
from google.colab import drive
drive.mount('/content/drive')
print('/content/drive/MyDrive')

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing the Articles data

import os
import pandas as pd
path = "/content/drive/MyDrive/ML/train.csv"


if os.path.exists(path):
    print("File exists")
else:
    print("File does not exist")


df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
print(df['article'])

In [101]:
# Important Arrays for Maintaining The Data and outputs
article = []
binArray = []
tfidf_vector_arr = []
entity = []
refehighlights = []
originalLen = []
newLenTfiDF = []
newLenBOW = []

In [ ]:
# Installing the NLTK Libarires
import nltk
nltk.download('all')

In [ ]:
# Appending the articles in the article array
# Same with the reference of Articles
article = []
refehighlights = []

for art in df['article']:
  article.append(art)
  

i = 0
for ref in df['highlights']:
  refehighlights.append(ref)

print(len(article) == len(refehighlights))

In [ ]:
# Importing Stop Words
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
stopwords

In [ ]:
# Importing Punctuation
from string import punctuation
punctuation = punctuation + '\n'
punctuation

In [14]:
# Using Spacy for tokenization
import spacy
nlp = spacy.load("en_core_web_sm")

In [15]:
# Counting the entities in the articles.
def count_entity(article,entities):
  # doc = nlp(article)
  # count_s = 0;
  CE = []
  sentences = nltk.sent_tokenize(article) # Using NLTK library for sentence tokenization.
  for sent in sentences:
    word_c = 0
    doc = nlp(sent) # Performing NLP using Spacy
    for ent in doc.ents:
      if(ent.label_ in entities):
        word_c+=1
    CE.append(word_c)
  return CE

In [16]:
# Calculating the sentence length
def sentence_length(article):
  sent_len = []
  sentences = nltk.sent_tokenize(article)
  for sent in sentences:
    word_c = 0
    for word in sent:
      word_c += 1
    sent_len.append(word_c)
  return sent_len

In [17]:
# Evaluating the Binary score method-1
# If entity found: Score: 1
# else: Score: 0
# With respect of sentnce

def count_binArray(article,entites):
  text = article
  final_BE1 = []
  f = 0
  sentences = nltk.sent_tokenize(text)
  for sent in sentences:
    f = 0
    doc = nlp(sent)
    for ent in doc.ents:
      if(ent.label_ in entites):
        final_BE1.append(1)
        f = 1
        break
    
    if(f==0):
      final_BE1.append(0)
      

  return final_BE1



In [18]:
# Evaluating the Binary score method-1
# If entity count > threshold: Score: 1
# else: Score: 0
# With respect of Article
def count_binArray2(article,entities):
  text = article
  # binDict = {}
  binArray = []

  sentences = nltk.sent_tokenize(article)
  article_ec = 0
  doc_art = nlp(article)
  for ent in doc_art.ents:
    if(ent.label_ in entities):
      article_ec+=1
  
  word_count_art = len(article.split(" "))

  threshold = article_ec/word_count_art # Calculation of Threshold

  for sent in sentences:
    sent_ec = 0
    doc = nlp(sent)
    for ent in doc.ents:
      if(ent.label_ in entities):
        sent_ec += 1
    
    score = sent_ec/len(sent.split(" "))
    if(score >= threshold):
      binArray.append(1)
    else:
      binArray.append(0)

  return binArray
    


In [19]:
# Calculating the count vectorizer for articles
def count_vectorizer(article):

  sentences = nltk.sent_tokenize(article)
  fina_CV = []
  for sent in sentences:
    count_dict = {}
    sum1 = 0
    for word in sent:
      if((word not in stopwords) and (word not in punctuation)):
        if(word in count_dict):
          count_dict[word] += 1
        else:
          count_dict[word] = 1
    sum1 = sum(count_dict.values())
    fina_CV.append(sum1)

  return fina_CV

      

In [20]:
# Calculating the TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

def TfIdf(article):
  tfidf_sen_scores = []
  # stop_words = set(stopwords.words("english"))
  # articles = [article]
  sentences = nltk.sent_tokenize(article)

  # Initialize a TF-IDF vectorizer
  tfidf_vectorizer = TfidfVectorizer()

  # Fit and transform the vectorizer to the list of articles
  tfidf = tfidf_vectorizer.fit_transform(sentences)

  feature_names = tfidf_vectorizer.get_feature_names_out() 

  tfidf_dicts = []
  for i in range(len(sentences)):
    tfidf_scores = tfidf[i].toarray()[0]
    tfidf_dict = dict(zip(feature_names, tfidf_scores))
    tfidf_dicts.append(tfidf_dict)

  
  for score_dict in tfidf_dicts:
    tfidf_sen_scores.append(sum(score_dict.values()))

  return tfidf_sen_scores 

In [21]:
# Filtering Global entites list with the help of the reference summary
def filter_entity(entity_types,text):
  entity = set()
  words = nltk.word_tokenize(text)
  tagged = nltk.pos_tag(words)
  entities = nltk.chunk.ne_chunk(tagged)
  for subtree in entities.subtrees():
    if subtree.label() in entity_types:
      entity.add(subtree.label())
  return entity

In [22]:
# Global Entites
entity_types = ['ORGANIZATION', 'PERSON', 'LOCATION', 'DATE', 'TIME']

In [ ]:
# Checking all the methods using article number-3
Samp_art = article[4]
highLight = refehighlights[4]

# Getting the entitites from reference summ
samp_tfidf = TfIdf(Samp_art)
samp_entity = filter_entity(entity_types,highLight)
samp_CE = count_entity(Samp_art,samp_entity)
samp_sen_len = sentence_length(Samp_art)
samp_bin_arr = count_binArray(Samp_art,samp_entity)
samp_bin_arr2 = count_binArray2(Samp_art,samp_entity)
print(len(samp_CE) == len(samp_sen_len))
print(samp_CE)
print(samp_bin_arr)
print(len(samp_CE))
print(len(samp_bin_arr))

In [ ]:
# Feature Extraction using all the methods
final_sent_CE = []
final_sent_len = []
final_sent_bin_arr = []
final_sent_bin_arr2 = []
final_count_vec = []
final_tfidf = []
final_sent_ids = []
i = 0
while(i<len(article)):
  Samp_art = article[i]
  highLight = refehighlights[i]

  # Getting the entitites from reference summ
  samp_entity = filter_entity(entity_types,highLight)
  samp_CE = count_entity(Samp_art,samp_entity)
  samp_sen_len = sentence_length(Samp_art)
  samp_bin_arr = count_binArray(Samp_art,samp_entity)
  samp_bin_arr2 = count_binArray2(Samp_art,samp_entity)
  samp_count_vec = count_vectorizer(Samp_art)
  samp_tfidf = TfIdf(Samp_art)

  # Index to each sentence

  tot_sent = len(nltk.sent_tokenize(article[i]))

  


  
  for CE in samp_CE:
    final_sent_CE.append(CE)

  for len_s in samp_sen_len:
    final_sent_len.append(len_s)

  for bin in samp_bin_arr:
    final_sent_bin_arr.append(bin)

  for bin2 in samp_bin_arr2:
    final_sent_bin_arr2.append(bin2)

  for cv in samp_count_vec:
    final_count_vec.append(cv)

  for tf in samp_tfidf:
    final_tfidf.append(tf)

  for k in range(tot_sent):
    final_sent_ids.append(i)

  i += 1

In [ ]:
# Selecting 15000 article from the whole dataset due to computational power limit.
final_sent_bin_arr1 = []
while(i<=15000):
  
  # final_sent_bin_arr1 = []
  Samp_text = article[i]
  highLight = refehighlights[i]
  samp_entity = filter_entity(entity_types,highLight)
  samp_bin_arr2 = count_binArray(Samp_text,samp_entity)


  for bin2 in samp_bin_arr2:
    final_sent_bin_arr1.append(bin2)

  i+=1

In [ ]:
print(len(final_sent_bin_arr1))

In [ ]:
print(len(final_sent_CE))
print(len(final_sent_len))
print(len(final_sent_bin_arr))
print(len(final_sent_bin_arr2))
print(len(final_count_vec))
print(len(final_tfidf))

In [24]:
#FROM HERE==========================================
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
# Creating the Final DataFrame to construct the final Dataset.
df_t = pd.DataFrame(list(zip(final_sent_ids,final_sent_CE,final_sent_len,final_count_vec,final_tfidf,final_sent_bin_arr,final_sent_bin_arr2)),columns = ['Article_Ids','Count_Entities','Sentence_Length','Count_Vectorizer','Tf-IDF_Vectorizer','Binary_Score1','Binary_Score2'])

In [ ]:
# Creating the CSV file
df_t.to_csv('/content/drive/MyDrive/Machine_Learning/output.csv')

In [ ]:
#from here====================================================================================
# Working with the saved CSV file to avoid the extraction of data every time.

import pandas as pd
#path = "/content/drive/MyDrive/machine_Learning/train.csv"

import os

path = "/content/drive/MyDrive/ML/file_10000.csv"

if os.path.exists(path):
    print("File exists")
else:
    print("File does not exist")


df_t = pd.read_csv(path)

#df_t = pd.DataFrame(list(zip(final_sent_CE,final_sent_len,final_count_vec,final_tfidf,final_sent_bin_arr,final_sent_bin_arr2)),columns = ['Count_Entities','Sentence_Length','Count_Vectorizer','Tf-IDF_Vectorizer','Binary_Score1','Binary_Score2'])

In [ ]:
import csv

column_1_name = "Count_Entities" # replace with the name of the first column you want to check
column_2_name = "Binary_Score1" # replace with the name of the second column you want to check

with open('/content/drive/MyDrive/Machine_Learning/output.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        column_1 = float(row[column_1_name])
        column_2 = float(row[column_2_name])
        if (column_1 <= 0 and column_2 > 0) or (column_1 > 0 and column_2 <= 0):
            print("Error: values in columns {} and {} are not consistent".format(column_1_name, column_2_name))


In [ ]:
df_t.head()

In [ ]:
df_t[df_t.Count_Entities == 1]

In [30]:
print(df_t.head())

   Article_Ids  Count_Entities  Sentence_Length  Count_Vectorizer  \
0            0               0                4                 3   
1            0               0               18                15   
2            0               0               12                10   
3            0               0               28                25   
4            0               0                3                 1   

   Tf-IDF_Vectorizer  Binary_Score1  Binary_Score2  
0           1.000000              0              0  
1           1.414214              0              0  
2           1.000000              0              0  
3           2.439751              0              0  
4           0.000000              0              0  


In [201]:
df_t['Binary_Score1'] =final_sent_bin_arr1

In [ ]:
df_t.to_csv('/content/drive/MyDrive/dataframe.csv', sep=',', index=False)

In [40]:
# Saperating the data for both the classes to plot the scatter plot
df0 = df_t[df_t.Binary_Score2 == 0]
df1 = df_t[df_t.Binary_Score2== 1] 

In [ ]:
plt.xlabel('Entities (words)')
plt.ylabel('Sentence length (words)')
plt.scatter(df0['Count_Entities'],df0['Sentence_Length'],color='green',marker='s')
plt.scatter(df1['Count_Entities'],df1['Sentence_Length'],color='blue',marker='^')

In [ ]:
plt.xlabel('Count Vectorizer (words)')
plt.ylabel('Sentence length (words)')
plt.scatter(df0['Count_Vectorizer'],df0['Sentence_Length'],color='green',marker='s')
plt.scatter(df1['Count_Vectorizer'],df1['Sentence_Length'],color='blue',marker='^')

<h3> This will Remain Constant for All models

Output will remain constant for all the models 

In [32]:
y = df_t.Binary_Score2

In [33]:
Score_dict = {}

<h3> Model-1 (x1,x2) </h3>

In [34]:
x12 = df_t.drop(['Article_Ids','Sentence_Length','Count_Vectorizer','Binary_Score1','Binary_Score2'],axis='columns')
x12.head()

,Count_Entities,Tf-IDF_Vectorizer
0,0,1.000000
1,0,1.414214
2,0,1.000000
3,0,2.439751
4,0,0.000000


In [35]:
x13 = df_t.drop(['Article_Ids','Sentence_Length','Tf-IDF_Vectorizer','Binary_Score1','Binary_Score2' ], axis='columns')
x13.head()

,Count_Entities,Count_Vectorizer
0,0,3
1,0,15
2,0,10
3,0,25
4,0,1


In [36]:
x14 = df_t.drop(['Article_Ids','Count_Vectorizer','Tf-IDF_Vectorizer','Binary_Score1','Binary_Score2'], axis='columns')
x14.head()

,Count_Entities,Sentence_Length
0,0,4
1,0,18
2,0,12
3,0,28
4,0,3


In [37]:
x24 = df_t.drop(['Article_Ids','Count_Vectorizer','Count_Entities','Binary_Score1','Binary_Score2'], axis='columns')
x24.head()

,Sentence_Length,Tf-IDF_Vectorizer
0,4,1.000000
1,18,1.414214
2,12,1.000000
3,28,2.439751
4,3,0.000000


In [38]:
x34 = df_t.drop(['Article_Ids','Tf-IDF_Vectorizer','Count_Entities','Binary_Score1','Binary_Score2'], axis='columns')
x34.head()

,Sentence_Length,Count_Vectorizer
0,4,3
1,18,15
2,12,10
3,28,25
4,3,1


In [39]:
x14 = df_t.drop(['Article_Ids','Tf-IDF_Vectorizer','Count_Entities','Binary_Score1','Binary_Score2'], axis='columns')
x34
# len(x34)

,Sentence_Length,Count_Vectorizer
0,4,3
1,18,15
2,12,10
3,28,25
4,3,1
...,...,...
995,12,10
996,28,25
997,3,1
998,10,8


In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Count_Entities')
plt.ylabel('Tf-IDF_Vectorizer')
plt.scatter(df0['Count_Entities'],df0['Tf-IDF_Vectorizer'],color='green',marker='s')
plt.scatter(df1['Count_Entities'],df1['Tf-IDF_Vectorizer'],color='blue',marker='^')

Splitting the dataset in to test-train for all the models

In [44]:
from sklearn.model_selection import train_test_split

x12_train,x12_test,y_train,y_test = train_test_split(x12,y,test_size=0.2)

In [45]:
x13_train,x13_test,y_train,y_test = train_test_split(x13,y,test_size=0.2)

In [46]:
x14_train,x14_test,y_train,y_test = train_test_split(x14,y,test_size=0.2)

In [47]:
x24_train,x24_test,y_train,y_test = train_test_split(x24,y,test_size=0.2)

In [48]:
x34_train,x34_test,y_train,y_test = train_test_split(x34,y,test_size=0.2)

<h3>Model with all the kernels

Model-1 (RBF)

In [49]:
model1_rbf = SVC()
model1_rbf.fit(x12_train,y_train)

SVC()

In [ ]:
Score_dict['Model1-rbf'] = model1_rbf.score(x12_test,y_test)
Score_dict['Model1-rbf']

Model-1 (Linear)

In [51]:
model1_linear = SVC(kernel='linear')
model1_linear.fit(x12_train,y_train)

SVC(kernel='linear')

In [ ]:
Score_dict['Model1-linear'] = model1_linear.score(x12_test,y_test)
Score_dict['Model1-linear']

Model-1 (Polynomail)

In [60]:
model1_polynomial = SVC(kernel='poly',degree=3)
model1_polynomial.fit(x12_train,y_train)

SVC(kernel='poly')

In [ ]:
Score_dict['Model1-polynomial'] = model1_polynomial.score(x12_test,y_test)
Score_dict['Model1-polynomial']

Model-1 (Sigmoid)

In [62]:
model1_sigmoid = SVC(kernel='sigmoid')
model1_sigmoid.fit(x12_train,y_train)

SVC(kernel='sigmoid')

In [ ]:
Score_dict['Model1-sigmoid'] = model1_sigmoid.score(x12_test,y_test)
Score_dict['Model1-sigmoid']

Model-2 (**Linear**)

In [64]:
model2_linear = SVC(kernel='linear')
model2_linear.fit(x13_train,y_train)

SVC(kernel='linear')

In [ ]:
Score_dict['Model2-linear'] = model2_linear.score(x13_test,y_test)
Score_dict['Model2-linear']

Model-2(rbf)

In [66]:
model2_rbf = SVC()
model2_rbf.fit(x13_train,y_train)

SVC()

In [ ]:
Score_dict['Model2-rbf'] = model2_rbf.score(x13_test,y_test)
Score_dict['Model2-rbf']

Model-2 (Polynomial)

In [68]:
model2_polynomial = SVC(kernel='poly',degree=3)
model2_polynomial.fit(x13_train,y_train)

SVC(kernel='poly')

In [ ]:
Score_dict['Model2-polynomial'] = model2_polynomial.score(x13_test,y_test)
Score_dict['Model2-polynomial']

Model-2 (Sigmoid)

In [70]:
model2_sigmoid = SVC(kernel='sigmoid')
model2_sigmoid.fit(x13_train,y_train)

SVC(kernel='sigmoid')

In [ ]:
Score_dict['Model2-sigmoid'] = model2_sigmoid.score(x13_test,y_test)
Score_dict['Model2-sigmoid']

Model-3 (Linear)

In [72]:
model3_linear = SVC(kernel='linear')
model3_linear.fit(x14_train,y_train)

SVC(kernel='linear')

In [ ]:
Score_dict['Model3-linear'] = model3_linear.score(x14_test,y_test)
Score_dict['Model3-linear']

Model-3 (rbf)

In [74]:
model3_rbf = SVC()
model3_rbf.fit(x14_train,y_train)

SVC()

In [ ]:
Score_dict['Model3-rbf'] = model3_rbf.score(x14_test,y_test)
Score_dict['Model3-rbf']

Model-3 (Polynomial)

In [75]:
model3_polynomial = SVC(kernel='poly',degree=3)
model3_polynomial.fit(x14_train,y_train)

SVC(kernel='poly')

In [ ]:
Score_dict['Model3-polynomial'] = model3_polynomial.score(x14_test,y_test)
Score_dict['Model3-polynomial']

Model-3 (Sigmoid)

In [78]:
model3_sigmoid = SVC(kernel='sigmoid')
model3_sigmoid.fit(x14_train,y_train)

SVC(kernel='sigmoid')

In [ ]:
Score_dict['Model3-sigmoid'] = model3_sigmoid.score(x14_test,y_test)
Score_dict['Model3-sigmoid']

Model-4 (Linear)

In [80]:
model4_linear = SVC(kernel='linear')
model4_linear.fit(x24_train,y_train)

SVC(kernel='linear')

In [ ]:
Score_dict['Model4-linear'] = model4_linear.score(x24_test,y_test)
Score_dict['Model4-linear']

Model-4 (rbf)

In [82]:
model4_rbf = SVC()
model4_rbf.fit(x24_train,y_train)

SVC()

In [ ]:
Score_dict['Model4-rbf'] = model4_rbf.score(x24_test,y_test)
Score_dict['Model4-rbf']

Model-4 (polynomial)

In [84]:
model4_polynomial = SVC(kernel='poly',degree=3)
model4_polynomial.fit(x24_train,y_train)

SVC(kernel='poly')

In [ ]:
Score_dict['Model4-polynomial'] = model4_polynomial.score(x24_test,y_test)
Score_dict['Model4-polynomial']

Model-4 (Sigmoid)

In [86]:
model4_sigmoid = SVC(kernel='sigmoid')
model4_sigmoid.fit(x24_train,y_train)

SVC(kernel='sigmoid')

In [ ]:
Score_dict['Model4-sigmoid'] = model4_sigmoid.score(x24_test,y_test)
Score_dict['Model4-sigmoid']

Model-5 (linear)

In [88]:
model5_linear = SVC(kernel='linear')
model5_linear.fit(x34_train,y_train)

SVC(kernel='linear')

In [ ]:
Score_dict['Model5-linear'] = model5_linear.score(x34_test,y_test)
Score_dict['Model5-linear']

Model-5 (rbf)

In [90]:
model5_rbf = SVC()
model5_rbf.fit(x34_train,y_train)

SVC()

In [ ]:
Score_dict['Model5-rbf'] = model5_rbf.score(x34_test,y_test)
Score_dict['Model5-rbf']

Model-5 (polynomial)

In [92]:
model5_polynomial = SVC(kernel='poly',degree=3)
model5_polynomial.fit(x34_train,y_train)

SVC(kernel='poly')

In [ ]:
Score_dict['Model5-polynomial'] = model5_polynomial.score(x34_test,y_test)
Score_dict['Model5-polynomial']

Model-5 (sigmoid)

In [94]:
model5_sigmoid = SVC(kernel='sigmoid')
model5_sigmoid.fit(x34_train,y_train)

SVC(kernel='sigmoid')

In [ ]:
Score_dict['Model5-sigmoid'] = model5_sigmoid.score(x34_test,y_test)
Score_dict['Model5-sigmoid']

In [96]:
#==========================here========================================
from sklearn.model_selection import GridSearchCV

param_grid = {'C':[0.1,1,10,100,1000],
              'gamma':[1,0.1,0.01,0.001,0.0001],
              'kernel':['rbf']}
grid = GridSearchCV(estimator=model1_rbf,param_grid=param_grid,cv=5)
grid.fit(x12_train,y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']})

In [97]:
best_model = grid.best_estimator_

In [ ]:
from sklearn.metrics import classification_report

y_pred = best_model.predict(x12_test)

print(classification_report(y_test,y_pred))